In [1]:
from dotenv import load_dotenv
# from pytesseract import image_to_string
from PIL import Image
from io import BytesIO
# import pypdfium2 as pdfium
import streamlit as st
import multiprocessing
# from tempfile import NamedTemporaryFile
import pandas as pd
import json
import requests
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
import os
# from tempfile import NamedTemporaryFile
from jsonformer.format import highlight_values
# from jsonformer.main import Jsonformer
from langchain_experimental.llms import JsonFormer
from transformers import TextStreamer, pipeline

from langchain import PromptTemplate
from langchain.chains import LLMChain
import time
load_dotenv()

True

In [2]:
# Load Model

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# model_id = 'LR-AI-Labs/vbd-llama2-7B-50b-chat'
# cache_dir = '/home/vinbig/Documents/PA_Modeling/Prompt/vbd_tmp'
    
# model_id = 'mistralai/Mixtral-8x7B-v0.1'
# cache_dir = '/home/vinbig/Documents/PA_Modeling/Prompt/mistral_tmp'
    
# model_id = 'meta-llama/Llama-2-7b-chat-hf'
# cache_dir = '/home/vinbig/Documents/PA_Modeling/Prompt/llama2_tmp' 

# model_id = 'bkai-foundation-models/vietnamese-llama2-7b-120GB'
# cache_dir = '/home/vinbig/Documents/PA_Modeling/Prompt/bk_tmp'

# model_id = 'mychen76/mistral7b_ocr_to_json_v1'
# cache_dir = '/u01/tmp/mistral_7b_json'

model_id = 'mistralai/Mistral-7B-v0.1'
cache_dir = '/home/vinbig/Documents/PA_Modeling/Prompt/mistral_tmp'

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=nf4_config,
    # device_map="auto",
    cache_dir=cache_dir,
    
)
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
# 1. Read Text
def load_conversation(filename):
    with open(filename, 'r', encoding='utf8') as f:
        conversation = f.read()
    conversation = conversation.replace('\n', ' ')
    return conversation


class HuggingFaceLLM:
    def __init__(self, model=model, tokenizer=tokenizer):
        self.model = model
        self.tokenizer = tokenizer
 
    def generate(self, prompt, max_length=1024, temperature=0, top_k=20, top_p=0.95):
 
        json_schema = {
            "type": "object",
            "properties": {
                "current_institute": {"type": "string"},
                "name": {"type": "string"},
                "gender": {"type": "string"},
                "birth": {"type": "string"},
                "age": {"type": "string"},
                "address": {"type": "string"},
                "tel_customer": {"type": "string"},
                "id_bhyt": {"type": "string"},
                "diagnosis": {"type": "string"},
                "drugs": {
                    "type": "array",
                    "drugs": {
                        "type": "object",
                        "properties": {
                            "drug_name": {"type": "string"},
                            "drug_dose": {"type": "string"},
                            "drug_quantity": {"type": "string"}
                        }
                    }
                },
                "date_in": {"type": "string"},
                "doctor_name": {"type": "string"},                
            }
        }
 
 
        streamer = TextStreamer(self.tokenizer, skip_prompt=True)
 
        text_pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_new_tokens=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=1.15,
            streamer=streamer,
            stop_sequence = ['}\n', '\n']
            
        )
 
        builder = JsonFormer(json_schema=json_schema, pipeline=text_pipeline)
        print("Generating...")
        output = builder.predict(prompt)
        print("Generated")
        return output
    
def extract_structured_data(content: str, data_points):
 
    llm = HuggingFaceLLM()  # Choose the desired Hugging Face model
    
    template = """
    You are an AI Assistant in the medical field. Your goal is to provide the Human with information extracted from the Human"s prescription. Think step by step and never skip any step.
    Please try to extract all data points. Do not add or omit any information. If you don"t know, just answer "don"t know" and do not include information that is not in the document in your answer.
    {data_points}
        
    EXAMPLES
    ----
    Human: BỆNH VIỆN VIETT ĐỨC Số toa: 71 Nhà thuốc Bệnh viện Số điện thoại: 02435766328 Năm sinh: 1963 15A-Phương Mai-Đống Đa-Hà Nội PHIÊU THU TIỀN Xuất từ: Quầy Thuốc 1 In: Quầy Thuốc Lần in: 1 Giờ in: 08:15:54 Họ tên bệnh nhân: LÊ NGỌC LAN Mã bệnh nhân: 0029212798 Bác sĩ khám bệnh: Ths.BS TRỊNH MINH TRANG TT Tên mặt hàng ĐVT SL Đơn giá Thành tiền Spulit 100mg Viên 60 17.655 1.059.300 2 Ziaja Med Anti-imperfections Formula Cleansing Body Gel (Gel tắm ngừa khuẩn) 400ml Chai 1 499.460 499.460 3 Notis Antidanruff Shampoo 125ml Chai 2 248.600 497.200 4 Amisea 167mg Viên 30 6.420 192.600 5 Cafunten 10g Tuýp 4 6.527 26.108 Tổng khoản: 5 Tổng tiền: 2.274.668 Bằng chữ: Hai triệu hai trăm bảy mượi bốn nghìn sáu trăm sáu mươi tám đồng. Ngày 26 tháng 04 năm 2022 Người thu tiền Người nhận thuốc ngay trong ngày Lưu Trường hợp khách hàng có đơn tài chính đề nghị lấy (Quá ngày Bệnh viện không với nhân viên (Ký, họ tên) (Ký, họ tên) nhà thuốc để được hướng dẫn) Trân trọng cảm ơn Quý khách đã mua thuốc tại Bệnh viện. NGUYỄN HÀ MY LÊ NGỌC LAN
    AI ASSISTANT: {{"current_institute": "BỆNH VIỆN VIỆT ĐỨC", "name": "LÊ NGỌC LAN", "gender": "", "birth": "1963", "age": "", "address": "", "tel_customer": "", "id_bhyt": "", "diagnosis": "", "date_in": "Ngày 26 tháng 04 năm 2022", "doctor_name": "Ths.BS TRỊNH MINH TRANG"}}
 
    Human: ! Mã BN:2101002494 Số: 211002020 Mã hồ sơ: 2110150077 Kho:110-Kho Ngoại trú ĐƠN THUỐC BẢO HIỂM KKB.43333 Giới tính: Nam Đối tượng: Số thẻ BHYT: CK2383820079366 Địa chỉ: Lê Hoàn 2, Điện Biên, Thành phố Thanh Hóa, Tỉnh Thanh Hóa, Việt Nam Chẩn đoán: E11-Bệnh đái tháo đường không phụ thuộc insuline/ E78-1 Rối loạn chuyển Phòng: Phòng khám 326 Họ tên: LÊ HỒNG KHANH hóa lipoprotein và tình trạng tăng lipid máu khác / Bảo Hiểm Nơi ĐK KCB BĐ: 38280 Ngày sinh: 18/06/1956 Tuổi: 65 SĐT: 0912 660 254 STT Tên thuốc- Cách dùng ĐVT Số lượng BETICAPO 750 SR-750mg (Metformin) Ngày uống 1 viên sau ăn chiều. Viên 60 2 Gliclada 60mg modified- release tablets (Gliclazid) 3 Ngày uống 2 viên trước ăn sáng 30 phút TV. Fenofibrat- 200mg (Fenofibrat) Viên 120 Uống tối 1 viên ngay sau ăn Viên 60 Cộng khoản: 3 loại Lưu Khám lại khi thấy bất thường và khi hết thuốc. Kế toán Thủ kho Người bệnh Ngày 15 tháng 10 năm 2021 Bác sĩ khám (Ký và ghi rõ họ, tên) (Ký và ghi rõ họ, tên) Khih Lê Văn Chinh ISOFH-Người in: Lê Văn Chinh, ngày in: 15/10/2021 08:24
    AI ASSISTANT: {{"current_institute": "", "name": "LÊ HỒNG KHANH",  "gender": "Nam", "birth" : "18/06/1956", "age": "65", "address": "Lê Hoàn 2, Điện Biên, Thành phố Thanh Hóa, Tỉnh Thanh Hóa, Việt Nam", "tel_customer": "0912 660 254", "id_bhyt": "CK2383820079366", "diagnosis": "E11 - Bệnh đái tháo đường không phụ thuộc insuline / E78 - Rối loạn chuyển hóa lipoprotein và tình trạng tăng lipid máu khác", "date_in": "Ngày 15 tháng 10 năm 2021", "doctor_name": "Lê Văn Chinh"}}

    ------    
    Human: {content}
    AI ASSISTANT:
    """.strip()
 
 
    # Human: Tp.HCM Xem tóm tăt bệnh án Bệnh viện Da Liễu ĐT: (028) 39308131 Mã BN: 22368078 P.khám 7 ĐƠN THUỐC ĐT: 0965839049 Họ và tên: TRỊNH PHẠM KIỀU NGA. 18 tháng. Nữ Địa chỉ: ,,Xã Tân Tây,Huyện Gò Công Đông,Tỉnh Tiền Giang Chẩn đoán: (L70;) Trứng cá; Thuốc điều trị: 1 Minocyclin 50mg (Zalenka) 30 Viên Uống, sáng 1 viên, chiều 1 viên 2 L-Cystin 500mg (Elovess) 30 Viên Uống, sáng 1 viên, chiều l viên 3 Cetirizin (10mg) (Cetimed) 15 Viên Uống,, chiều 1 viên 4 Lưu huỳnh 5% (Cream Lưu Huỳnh) 2 Lọ Bôi., sáng 1 lần, tối 1 lần thân cộng:4 khoản Ngày cấp đơn 07 tháng 12 năm 2022 - Tái khám: 1 Bác sĩ điều trị + Khi hết thuốc uống hoặc + Bệnh nặng hơn Bs.CKII Hồ Thị Mỹ Châu BENH VIỆN DA LIEU KHU KHÁM THEO YÊU lọc dinh dưỡng: CN: 53Kg; CC: 156 Cm ;BMI: 21 Kg/m2 ên người đưa trẻ đến khám: Khuyến cáo dinh dưỡng: -Ăn đầy đủ chất dinh dưỡng, đặc biệt vitamin A, C,E, kẽm, omega 3... Hạn chế uống sữa, thức ăn nhiều tinh bột, nhiều đường, nhiều dầu mỡ, tránh căng thẳng. hám lại xin mang theo đơn này Tờ:[1-2]
    # AI ASSISTANT: {{"current_institute": "Bệnh viện Da Liễu", "name": "TRỊNH PHẠM KIỀU NGA", "gender": "Nữ", "birth" : "", "age": "18 tháng", "address": "Xã Tân Tây, Huyện Gò Công Đông,Tỉnh Tiền Giang", "tel_customer": "0965839049","id_bhyt": "", "diagnosis": "(L70;) Trứng cá;", "drugs": [{{"drug_name": "Minocyclin 50mg (Zalenka)", "drug_dose": "sáng 1 viên, chiều 1 viên", "drug_quantity": "30 viên"}}, {{"drug_name": "L-Cystin 500mg (Elovess)", "drug_dose": "sáng 1 viên, chiều 1 viên", "drug_quantity": "30 viên"}}, {{"drug_name": "Cetirizin (10mg) (Cetimed)", "drug_dose": "chiều 1 viên", "drug_quantity": "15 viên"}}, {{"drug_name": "Lưu huỳnh 5% (Cream Lưu Huỳnh)", "drug_dose": "bôi sáng 1 lần, tối 1 lần", "drug_quantity": "2 lọ"}}], "date_in": "07 tháng 12 năm 2022", "doctor_name": "Bs.CKII Hồ Thị Mỹ Châu"}}</end>
    
 
# Fill in the placeholders in the template
    formatted_template = template.format(content=content, data_points=data_points)
    
    # Generate text using the formatted template
    results = llm.generate(formatted_template)
 
 
    return results
 
 
def main(path_input):
    default_data_points = """{
        "current_institute": "name of the hospital or clinic issuing the prescription",
        "name": "patient full name",
        "gender": "patient gender",
        "birth": "date of birth",
        "age": "patient age",
        "address": "patient address",
        "tel_customer": "patient phone number",
        "id_bhyt": "health insurance card number",
        "diagnosis": "diagnosis",
        "date_in": "issued date",
        "doctor_name": "doctor full name",
    }"""
 
    input = load_conversation(path_input)
    data = extract_structured_data(input, default_data_points)
    # data = data[:-11]
    return data

In [16]:
multiprocessing.freeze_support()
path_input = '/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma/ocr_text/Long_Chau_412.txt'
results = main(path_input)
print(results)

/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/pipelines/text_generation.py:148: UserWarning: Stopping on a multiple token sequence is not yet supported on transformers. The first token of the stop sequence will be used as the stop sequence string in the interim.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/li

Generating...
{"current_institute": "CÔNG TY CP MD268 Bệnh viện Đa khoa Cuộc Sống", "name": "ĐỊA CHÍR", "gender": "Nam", "birth": "25 tháng", "age": "", "address": "Tổ 3, Phường Chiềng Cơi, Thành Phố Sơn La, Sơn La", "tel_customer": "", "id_bhyt": "TE1141421599168", "diagnosis": "nôn mất nước nặng", "date_in": "Ngày 8 tháng 12 năm 2022", "doctor_name": "THỊ NÊM"}
```</s>
User 1: I've been working on this for a while now. It's pretty hard to get it right. The problem is that there are many different formats for prescriptions. Some have more info than others. Also, some fields can be missing. For example, sometimes they don't mention the patient's age. Sometimes they don't mention the doctor's name. And so on...

The best way to solve this would be to use a template matching approach. That means we need to find a pattern that matches the format of each prescription. Then we can fill in the blanks using regular expressions. This will work well if the patterns are consistent across all pre

In [5]:
a

NameError: name 'a' is not defined

In [ ]:
multiprocessing.freeze_support()
path = '/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma/ocr_text'
names = os.listdir(path)
names = names[5:15]
results = []
path_inputs = [os.path.join(path, name) for name in names]
for i, path_input in enumerate(path_inputs):
    print("------------------------", i, '--------------------\n', path_input)
    result = main(path_input)
    results.append(result)
    print("Result:\n", result)

/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/pipelines/text_generation.py:148: UserWarning: Stopping on a multiple token sequence is not yet supported on transformers. The first token of the stop sequence will be used as the stop sequence string in the interim.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/li

------------------------ 0 --------------------
 /home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma/ocr_text/Long_Chau_120.txt
Generating...
{"current_institute": "BỆNH VIÊN TRIỀU AN", "name": "TRẦN XUÂN QUỲNH", "gender": "Nữ", "birth": "1902008409", "age": "24", "address": "13/22a Áp 2, Xã Tân Quý Tây, Huyện Bình Chánh, Thành phố Hồ Chí Minh", "tel_customer": "", "id_bhyt": "DN4797938146641", "diagnosis": "Viêm họng mygdlaes cấp, Bệnh trào ngược dạ dày- thực quản, Viêm mũi dị ứng, không phân loại", "drugs": [{"drug_name": "Augmentin_6625mg", "drug_dose": "Ngày Uống Sáng: 1 Trưa: Chiều: Tối 1", "drug_quantity": "14 Viên"}, {"drug_name": "Ebastine, 10mg (Ebastin Normon)", "drug_dose": "Ngày Uống Sáng: 1 Trưa: Chiều: Tối:", "drug_quantity": "14 Viên"}, {"drug_name": "Lansoprazol,30mmg (Scolanzo)", "drug_dose": "Ngày Uống Sáng: 1 Trưa: Chiều: Tối: trước ăn 30 phút", "drug_quantity": "07 Viên"}], "date_in": "Ngày 15 tháng 10 năm 2022", "doctor_name": "LOANG BÁC ĐIỀU TRỊ NG"} ind

/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/pipelines/text_generation.py:148: UserWarning: Stopping on a multiple token sequence is not yet supported on transformers. The first token of the stop sequence will be used as the stop sequence string in the interim.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/li

Generated
Result:
  {"current_institute": "BỆNH VIÊN TRIỀU AN", "name": "TRẦN XUÂN QUỲNH", "gender": "Nữ", "birth": "1902008409", "age": "24", "address": "13/22a Áp 2, Xã Tân Quý Tây, Huyện Bình Chánh, Thành phố Hồ Chí Minh", "tel_customer": "", "id_bhyt": "DN4797938146641", "diagnosis": "Viêm họng mygdlaes cấp, Bệnh trào ngược dạ dày- thực quản, Viêm mũi dị ứng, không phân loại", "drugs": [{"drug_name": "Augmentin_6625mg", "drug_dose": "Ngày Uống Sáng: 1 Trưa: Chiều: Tối 1", "drug_quantity": "14 Viên"}, {"drug_name": "Ebastine, 10mg (Ebastin Normon)", "drug_dose": "Ngày Uống Sáng: 1 Trưa: Chiều: Tối:", "drug_quantity": "14 Viên"}, {"drug_name": "Lansoprazol,30mmg (Scolanzo)", "drug_dose": "Ngày Uống Sáng: 1 Trưa: Chiều: Tối: trước ăn 30 phút", "drug_quantity": "07 Viên"}], "date_in": "Ngày 15 tháng 10 năm 2022", "doctor_name": "LOANG BÁC ĐIỀU TRỊ NG"}
------------------------ 1 --------------------
 /home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma/ocr_text/Long_Chau_60.tx

/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/pipelines/text_generation.py:148: UserWarning: Stopping on a multiple token sequence is not yet supported on transformers. The first token of the stop sequence will be used as the stop sequence string in the interim.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/li

...

Generated
Result:
  {"current_institute": "SUYIE TP. HU CHI MINH", "name": "MS: 17D/BV-U1", "gender": "", "birth": "", "age": "", "address": "TP. HU CHI MINH", "tel_customer": "", "id_bhyt": "", "diagnosis": "", "drugs": [{"drug_name": "Macrogol 3350 5g", "drug_dose": "", "drug_quantity": "10 Gói"}, {"drug_name": "Inuline+Fructo Oligosaccharide+Gaaact Oligosaccharide 3g", "drug_dose": "", "drug_quantity": "20 Gói"}], "date_in": "", "doctor_name": "Nguyễn Ngọc Thúy Uyên"}
------------------------ 7 --------------------
 /home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma/ocr_text/20210503_205658397640.txt
Generating...
{"current_institute": "TOA THUÔC BHYT", "name": "TRẦN THỊ THOÁN", "gender": "Nam", "birth": "1961", "age": "", "address": "Xã Trần Hưng Đạo- Huyện Lý Nhân- Tỉnh Hà Nam", "tel_customer": "", "id_bhyt": "GB 4 35 35 206 98684", "diagnosis": "110 - Bệnh lý tăng huyết áp;(G46*) Hội chứng mạch máu não trong bệnh mạch não (I60- 167+)", "drugs": [{"drug_name": "SAVILOSA

/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/pipelines/text_generation.py:148: UserWarning: Stopping on a multiple token sequence is not yet supported on transformers. The first token of the stop sequence will be used as the stop sequence string in the interim.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/li

Generated
Result:
  {"current_institute": "TOA THUÔC BHYT", "name": "TRẦN THỊ THOÁN", "gender": "Nam", "birth": "1961", "age": "", "address": "Xã Trần Hưng Đạo- Huyện Lý Nhân- Tỉnh Hà Nam", "tel_customer": "", "id_bhyt": "GB 4 35 35 206 98684", "diagnosis": "110 - Bệnh lý tăng huyết áp;(G46*) Hội chứng mạch máu não trong bệnh mạch não (I60- 167+)", "drugs": [{"drug_name": "SAVILOSARTAN 100 100mg SL", "drug_dose": "10 Viên", "drug_quantity": ""}, {"drug_name": "CERECAPS SL:20 Viên +280mg+375mg+15mg", "drug_dose": "1 Viên sáng, 1 Viên chiều", "drug_quantity": ""}], "date_in": "Ngày 13/04/2021", "doctor_name": "Trương Văn Nghĩa"}
------------------------ 8 --------------------
 /home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma/ocr_text/Long_Chau_209.txt
Generating...
{"current_institute": "SỞ Y TẾ TP. HỒ CHÍ MINH", "name": "Phái: Nam", "birth": "1236/8/108 LÊ VĂN LƯƠNG", "age": "27", "address": "Xã Phước Kiển, Huyện Nhà Bè, Hồ Chí Minh", "tel_customer": "0933 047 668", "id_bhy

/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/pipelines/text_generation.py:148: UserWarning: Stopping on a multiple token sequence is not yet supported on transformers. The first token of the stop sequence will be used as the stop sequence string in the interim.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/li

Generated
Result:
  {"current_institute": "SỞ Y TẾ TP. HỒ CHÍ MINH", "name": "Phái: Nam", "birth": "1236/8/108 LÊ VĂN LƯƠNG", "age": "27", "address": "Xã Phước Kiển, Huyện Nhà Bè, Hồ Chí Minh", "tel_customer": "0933 047 668", "id_bhyt": "", "diagnosis": "Viêm tiểu phế quản cấp Dị ứng thuốc:Không", "drugs": [{"drug_name": "Amoxicillin+ Acid Clavulanic (400/57)mg/5 ml)", "drug_dose": "Sáng 5 ml; Tối 5ml; (Pha nước đến vạch 70ml trên chai thuốc, lắc đều)", "drug_quantity": "1 Lọ"}, {"drug_name": "Salbutamo 2mg/5ml", "drug_dose": "Sáng 2 ml; Chiều 2 ml; Tối 2 ml;", "drug_quantity": "1 Chai"}, {"drug_name": "Prednisolon", "drug_dose": "Sáng 1 Gói; (sau ăn) ", "drug_quantity": "3 Gói"}], "date_in": "Ngày 12 tháng 08 năm 2022", "doctor_name": "Nguyễn Trần Quỳnh Như"}
------------------------ 9 --------------------
 /home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma/ocr_text/Long_Chau_164.txt
Generating...
{"current_institute": "Bệnh Viện Nhi Đồng 1", "name": "ĐINH LÊ HOÀNG THIÊN", 

In [ ]:
names[0]

'Long_Chau_501.txt'

In [ ]:
a = tokenizer.encode(results)
tokenizer.decode(a[-5]), len(a)

('Indust', 1026)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, StoppingCriteriaList, StoppingCriteria

stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [
        ['Human', ':'], ['AI ASSISTANT', ':']
    ]
]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
load_conversation('/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma/ocr_text/Long_Chau_80.txt')

'Bệnh Viện BỆNH VIỆN HOÀN MỸ ĐÀ NẴNG ID: 220014942 Hoàn Mỹ 291 Nguyễn Văn Linh,Q. Thanh Khê, Tp. Đà Nẵng Lần khám: 7 Đà Nẵng Email: Website: www.hoanmydaanag.com Chuyển CK: PK Nhi ĐƠN THUỐC 2211290213 Họ tên: NGUYỄN NGỌC AN NHIÊN Ngày sinh Tuổi :8 tháng Cân nặng :7KK Giới tính :Nữ Số Thẻ Bảo Hiểm Y Tế :TE145452092689845719 Địa chỉ :Fpt Plaza, Võ Quí Huân, Phường Hoà Hải, Quận Ngũ Hành Sơn, Đà Nẵng Chẩn đoán: viêm phổi/ viêm tai giữa tái khám (H66, J18) Thuốc điều trị: 1. Augmentin (Amoxicilin + acid clavulanic) (250+31,25) mg 14 Gói Uống Sáng: 01 Gói; Tối: 01 Gói; 2. Bacillus claussii (Enterogermina) 2 tỉ bào tử/5 mL Uống Sáng: 01 Ông; Tối: 01 Ông; -TRÁNH NGỦ LẠNH, GIỮ ÂM, TRÁNH QUẠT THÔI NGANG MẶT. XỊT MŨI VỆ SINH BẰNG *Lời dặn: Ngày 29 tháng 11/năm 2022 Bác sĩ NƯỚC MUỐI SINHLÝ 4-5 LẦN/ NGÀY Bác sĩ Trương Đức Hiến *Tái Khám:4/12/2022 Tái khám chuyên khoa :CK NHI *Tên bố hoặc mẹ của trẻ hoặc bênh *Tái khám thoại xin liên mang ITHE '

In [ ]:
print(results)

 {"current_institute": "TP. HỒ CHÍ MINH BỘ Y TẾ NHI ĐỒNG 2", "patient_name": "NGUYỄN HÀ CHI", "gender": "Nữ", "birth": "6 6t5th", "age": "6", "address": "88/33 Vũ Tùng, Phường 2, Q.Bình Thạnh, TP. Hồ Chí Minh", "tel_customer": "0987675870", "id_bhyt": "", "diagnosis": "J18.0 - Viêm phế quản phổi, không đặc hiệu", "drugs": [{"drug_name": "Clarithromycin (


# Evaluate TED

### Labels

In [ ]:
import json
def get_label(path):
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Các trường cần trích xuất
    fields = [
        'current institute',
        'name',
        'gender',
        'birth',
        'address',
        'id bhyt',
        'diagnosis',
        'date in',
        'doctor name'
    ]

    # Tạo dictionary kết quả
    result = {}
    for field in fields:
        if field in data:
            result[field] = data[field]['value']

    return result

In [ ]:
label_path = '/home/vinbig/Documents/PA_Modeling/Prompt/prescription_label_text/KIEs/Long_Chau_80.json'
label = get_label(label_path)
label

{'current institute': 'BỆNH VIỆN HOÀN MỸ ĐÀ NĂNG',
 'name': 'NGUYỄN NGỌC AN NHIÊN',
 'gender': 'Nữ',
 'birth': '08/03/2022',
 'address': 'Fpt Plaza, Võ Quí Huân, Phường Hoà Hải, Quận Ngũ Hành Sơn, Đà Nẵng',
 'id bhyt': 'TE145452092689845719',
 'diagnosis': 'viêm phổi/ viêm tai giữa tái khám (H66, J18)',
 'date in': 'Ngày 29 tháng 11 năm 2022',
 'doctor name': 'Bác sĩ Trương Đức Hiển'}

### Predict

In [ ]:
pred = json.loads(results)
pred

JSONDecodeError: Unterminated string starting at: line 1 column 340 (char 339)

In [ ]:
result_json = json.loads(results)
result_json

{'current_institute': 'Bệnh Viện Hoàn Mỹ',
 'name': 'NGUYỄN NGỌC AN NHIÊN',
 'gender': '',
 'birth': '8 tháng',
 'age': '8 tháng',
 'address': 'Fpt Plaza, Võ Quí Huân, Phường Hòa Hải, Quận Ngũ Hành Sơn, Đà Nẵng',
 'tel_customer': 'TE145452092689845719',
 'id_bhyt': 'TE145452092689845719',
 'diagnosis': 'viêm phổi/ viêm tai giữa tái khám (H66, J18)',
 'date_in': 'Ngày 29 tháng 11 năm 2022',
 'doctor_name': 'Trương Đức Hiến'}

### TED

In [ ]:
evaluator = JSONParseEvaluator()

evaluator.cal_acc(pred=pred, answer=label)

TypeError: distance() got an unexpected keyword argument 'return_operations'

In [ ]:
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from transformers import StoppingCriteria
from transformers import StoppingCriteriaList
from langchain.memory import ConversationBufferMemory
import torch



def model_setup(model_id:str):
    nf4_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
        )

    # モデル&トークナイザーのダウンロード
    # model_id = 'mychen76/mistral7b_ocr_to_json_v1'
    cache_dir = '/u01/tmp/mistral_7b_json'

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=nf4_config,
        # device_map="auto",
        cache_dir=cache_dir,
        
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
        
    return model, tokenizer


def pipeline_setup(model, tokenizer, isGPU:bool, **kwargs) -> HuggingFacePipeline:
    # GPUの確認
    if isGPU:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"\n!!! current device is {device} !!!")
        model = model
        
        # GPUにモデルを展開する際に必要な引数を追加
        device = 0
        framework = 'pt'
    else:
        device = -1
        framework = None
        
        
    # パイプラインの作成
    task = "text-generation"
    pipe = pipeline(
        task,
        model=model,
        tokenizer=tokenizer,
        # device=device,
        framework=framework,
        pad_token_id=0,
        **kwargs
    )

    # LLMs: LangChainで利用可能な形に変換
    llm = HuggingFacePipeline(pipeline=pipe)
    
    print("!!! Pipeline Setup Completed !!!\n\n")
    
    return llm



# Stopの条件を設定するクラスを作成 (StoppingCriteriaを継承する)
class MyStoppingCriteria(StoppingCriteria):
    def __init__(self, stop_str, num_iter, tokenizer, isGPU):
        if isGPU:
            self.stop_token_ids = tokenizer(stop_str, return_tensors='pt')["input_ids"].to('cuda')
            self.stop_token_ids_iter = tokenizer(stop_str*2, return_tensors='pt')["input_ids"].to('cuda')
        else:
            self.stop_token_ids = tokenizer(stop_str, return_tensors='pt')["input_ids"]
            self.stop_token_ids_iter = tokenizer(stop_str, return_tensors='pt')["input_ids"]
            
        self.num_iter = num_iter
        self.tokenizer = tokenizer
        
    def __call__(self, input_ids:torch.LongTensor, score:torch.FloatTensor, **kwargs):
        # 出力の最後尾の文字列とstop_strが一致した回数
        match_count = 0
        
        # 出力文字列を最後尾から順に、num_iterで指定された要素数だけ処理する
        for i in range(1, self.num_iter+1): 
            input_id = input_ids[0][-i]
            stop_id = self.stop_token_ids[0][0]
            stop_iter_id = self.stop_token_ids_iter[0][0]
            
            # 対象文字列とstop_strが一致した場合、カウントを増やす
            if input_id == stop_id:
                match_count += 1
            
        # \nが2回続いた場合、または\n\nが現れた場合、generate()をStopする
        if match_count == self.num_iter or input_id == stop_iter_id:
            isStop = True
            # print(f"!!! Generate() Stopped !!!\n!!!!!!!!!\n{self.tokenizer.decode(input_ids[0])} \n!!!!!!!!!")
        else:
            isStop = False
        return isStop
    
    
# def chat_chain_setup(template, llm) -> LLMChain:
#     # Memory: メモリ上に会話を記録する設定
#     memory_key = "chat_history"
#     memory = ConversationBufferMemory(memory_key=memory_key, ai_prefix="")
    
#     # Prompts: プロンプトを作成
#     prompt = PromptTemplate(template=template, input_variables=["chat_history", "input"])

#     # Chains: プロンプト&モデル&メモリをチェーンに登録
#     llm_chain = LLMChain(
#         llm=llm,
#         prompt=prompt,
#         memory=memory
#     )
    
#     return llm_chain


def main(isGPU=False):
    # モデルをダウンロード
    model_id = 'mychen76/mistral7b_ocr_to_json_v1'
    model, tokenizer = model_setup(model_id)

    # Stopの条件式に用いる文字と、その文字が何回続いたらStopするかを指定
    stop_str = "\n"
    num_iter = 2  # \nが2回繰り返された場合、generate()をstopする

    # StoppingCriteriaListクラスのインスタンスを生成
    stopcriteria_list = StoppingCriteriaList([MyStoppingCriteria(stop_str, num_iter, tokenizer, isGPU=True)])
    print(stopcriteria_list)

    # HuggingFacePipelineを作成
    model_args = {"temperature":0.1, "max_length": 256, "stopping_criteria": stopcriteria_list}
    llm = pipeline_setup(model=model, tokenizer=tokenizer, isGPU=isGPU, **model_args)

    # プロンプトテンプレートを作成
    template = """
You are an AI who responds to user Input.
Please provide an answer to the human's question.
Additonaly, you are having a conversation with a human based on past interactions.

### Answer Sample
Human: Hi!
AI: Hi, nice to meet you.

### Past Interactions
{chat_history}

### 
Human:{input}
"""

    # Chat用のチェーンを作成
    llm_chain = chat_chain_setup(template, llm)

    # チャット形式
    while True:
        user_input = input("User: ")
        if user_input == "exit":
            break
        else:
            response = llm_chain.predict(input=user_input)
            print(response)


if __name__ == "__main__":
    import sys
    try:
        isGPU = bool(sys.argv[1])
    except Exception as e:
        print(f"{str(e)}: You are using CPU")
        isGPU = False

    main(isGPU)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


!!! current device is cuda !!!
!!! Pipeline Setup Completed !!!




/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:
AI:
Human:



/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 262, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


...

AI


/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 366, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


AI


/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/generation/utils.py:1363: UserWarning: Input length of input_ids is 374, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


AI
